In [1]:
import pandas as pd
import numpy as np

In [2]:
#Import data

#get confussion matrix
cm = pd.read_csv("Family300320/confussionMatrix.csv",header=0,sep='\t',index_col=0)
## Import genomes vs viphogs matrix for family
reducedMatrix = pd.read_csv("Family300320/reducedViphogsMatrix.csv",header=0,sep='\t')
print reducedMatrix.shape
## Import list of important viphogs
importanViphogs = []
fr = open("Family300320/importantViphogs.minNumberBestScore.csv")
for line in fr:
    importanViphogs.append(line.strip('\n'))
## Reduce the matrix to only the important viphogs
reducedMatrix = reducedMatrix[importanViphogs+['Labels']]
print reducedMatrix.shape 
## Import dictionary of labels
labelsDict = np.load("Family/trainTestLabelsDictFamilyAll.npy").item()

(11978, 11329)
(11978, 389)


In [4]:
#get labels with good classification scores (>80%)
hsi = [] #hsi := high score indexes
i = 0
for score in np.diag(np.matrix(cm.as_matrix())):
    if score >= 0.8:
        hsi.append(i)
    i += 1
hsl = cm.index[hsi] #hsl := high score labels
print len(hsl)
print len(labelsDict)

80
83


In [5]:
#Get signature genomes per label
sl = [] #sl := signatures per label
n = [] #m := intersection between sl and viphogs of the other labels. If the interscetion is not empty that could compromise the "signature character" of the signatures
for label in hsl:
    #get signature viphogs for the actual label
    aux = reducedMatrix[reducedMatrix["Labels"]==labelsDict[label]].copy()
    aux.drop(["Labels"],axis=1,inplace=True)
    aux = aux.T[aux.sum(axis=0)>0].T.sum(axis=0).to_dict()#get the number of genomes that has a viphog
    sl.append(aux)
    #get viphogs present in other labels
    aux2 = reducedMatrix[reducedMatrix["Labels"]!=labelsDict[label]].copy()
    aux2.drop(["Labels"],axis=1,inplace=True)
    aux2 = aux2.T[aux2.sum(axis=0)>0].T.sum(axis=0).to_dict()#get the number of genomes that has a viphog
    #get the difference between genomes that belong to the label and genomes that don't belong
    if len((set(aux.keys())&set(aux2.keys())))>0:
        aux3 = {}
        for viphog in list((set(aux.keys())&set(aux2.keys()))):
            aux3[viphog] = aux2[viphog]
        n.append(aux3)
    else:
        n.append({})     

In [6]:
#Depurate signature viphogs
sld = [] #sld := signatures per label depurated
i = 0
for s in n: #s := set
    if len(s) > 0:
        #Depurate
        aux = {}
        for viphog in sl[i]:
            if viphog in s:
                #Depurate the viphog as genomes of other labels also have it
                if sl[i][viphog]/(s[viphog]+sl[i][viphog]) >= 0.8:
                    aux[viphog] = sl[i][viphog]
                else:
                    #don't include the viphog as from all the genomes that have that viphog,
                    #less than 80% genome belong to the actual label
                    continue 
            else:
                aux[viphog] = sl[i][viphog]
        sld.append(aux)
    else:
        sld.append(sl[i])
    i += 1

In [7]:
fw = open("Family300320/importantViphogs.minNumberBestScore.withTaxonomy.csv","w")
i = 0
for signatures in sld:
    for viphog in signatures:
        fw.write(viphog+'\t'+hsl[i]+'\n')
    i += 1
fw.close()  